# Agents

```{include} ../warning.md
```

AutoGen AgentChat provides a set of preset Agents, each with variations in how an agent might respond to messages.
All agents share the following attributes and methods:

- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.name`: The unique name of the agent.
- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.description`: The description of the agent in text.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run`: Run with a given task and produce a {py:class}`~autogen_agentchat.base.TaskResult`.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run_stream`: Run with a given task and produce an iterator of {py:class}`~autogen_agentchat.messages.AgentMessage` that ends with a {py:class}`~autogen_agentchat.base.TaskResult`.


## Assistant Agent

{py:class}`~autogen_agentchat.agents.AssistantAgent` is a built-in agent that
is using the ReAct pattern to generate responses with ability to use tools.

In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models import OpenAIChatCompletionClient


# Define a tool that searches the web for information.
async def web_search(query: str) -> str:
    """Find information on the web"""
    return "AutoGen is a programming framework for building multi-agent applications."


# Create an agent that uses the OpenAI GPT-4o model.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)
agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    tools=[web_search],
    system_message="Use tools to solve tasks.",
)

We can call the {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run` 
method to run the agent with a given task.

In [6]:
async def assistant_run() -> None:
    # Run the agent with a given task.
    result = await agent.run(task="Find information on AutoGen")
    print(result)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Find information on AutoGen'), ToolCallMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=82, completion_tokens=15), content=[FunctionCall(id='call_fNG1y1aEfDOVDMwWLGE83YXk', arguments='{"query":"AutoGen"}', name='web_search')]), ToolCallResultMessage(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_fNG1y1aEfDOVDMwWLGE83YXk')]), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=92, completion_tokens=14), content='AutoGen is a programming framework designed for creating multi-agent applications.')], stop_reason=None)


The call to the {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run` method
returns a {py:class}`~autogen_agentchat.base.TaskResult` with a list of messages
generated by the agent.

### Stream Messages

We can also stream each message as it is generated by the agent by using the
{py:meth}`~autogen_agentchat.agents.BaseChatAgent.run_stream` method.

In [ ]:
async def assistant_run_stream() -> None:
    # Run the agent with a given task and stream the response.
    async for message in agent.run_stream(task="Find information on AutoGen"):
        print(message)


# Use asyncio.run(assistant_run_stream()) when running in a script.
await assistant_run_stream()

source='user' models_usage=None content='Find information on AutoGen'
source='assistant' models_usage=RequestUsage(prompt_tokens=82, completion_tokens=15) content=[FunctionCall(id='call_0M1jNwCWR6sTnCxI4FC1Y1KA', arguments='{"query":"AutoGen"}', name='web_search')]
source='assistant' models_usage=None content=[FunctionExecutionResult(content='AutoGen is a programming framework for building multi-agent applications.', call_id='call_0M1jNwCWR6sTnCxI4FC1Y1KA')]
source='assistant' models_usage=RequestUsage(prompt_tokens=92, completion_tokens=16) content='AutoGen is a programming framework designed for the development of multi-agent applications.'
TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Find information on AutoGen'), ToolCallMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=82, completion_tokens=15), content=[FunctionCall(id='call_0M1jNwCWR6sTnCxI4FC1Y1KA', arguments='{"query":"AutoGen"}', name='web_search')]), ToolCallResultMessage(sour

The {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run_stream` method
returns an asynchronous generator that yields each message generated by the agent,
and the final task result as the last item.

From the messages, you can see the assistant agent used the `web_search` tool to
search for information and responded using the search results.

### Understanding Tool Calling

Large Language Models (LLMs) are typically limited to generating text or code responses. However, many complex tasks benefit from the ability to use external tools that perform specific actions, such as fetching data from APIs or databases.

To address this limitation, modern LLMs can now accept a list of available tool schemas (descriptions of tools and their arguments) and generate a tool call message. This capability is known as **Tool Calling** or **Function Calling** and is becoming a popular pattern in building intelligent agent-based applications.

For more information on tool calling, refer to the documentation from [OpenAI](https://platform.openai.com/docs/guides/function-calling) and [Anthropic](https://docs.anthropic.com/en/docs/build-with-claude/tool-use).

<!-- ## CodingAssistantAgent

Generates responses (text and code) using an LLM upon receipt of a message. It takes a `system_message` argument that defines or sets the tone for how the agent's LLM should respond. 

```python

writing_assistant_agent = CodingAssistantAgent(
    name="writing_assistant_agent",
    system_message="You are a helpful assistant that solve tasks by generating text responses and code.",
    model_client=model_client,
)
`

We can explore or test the behavior of the agent by sending a message to it using the  {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`  method. 

```python
result = await writing_assistant_agent.on_messages(
    messages=[
        TextMessage(content="What is the weather right now in France?", source="user"),
    ],
    cancellation_token=CancellationToken(),
)
print(result) -->

## CodeExecutorAgent

The {py:class}`~autogen_agentchat.agents.CodeExecutorAgent`
preset extracts and executes code snippets found in received messages and returns the output. It is typically used within a team with another agent that generates code snippets to be executed.

```{note}
It is recommended that the {py:class}`~autogen_agentchat.agents.CodeExecutorAgent` agent
uses a Docker container to execute code. This ensures that model-generated code is executed in an isolated environment. To use Docker, your environment must have Docker installed and running. 
Follow the installation instructions for [Docker](https://docs.docker.com/get-docker/).
```

In this example, we show how to set up a {py:class}`~autogen_agentchat.agents.CodeExecutorAgent` agent that uses the 
{py:class}`~autogen_ext.code_executors.DockerCommandLineCodeExecutor` 
to execute code snippets in a Docker container. The `work_dir` parameter indicates where all executed files are first saved locally before being executed in the Docker container.

In [ ]:
from autogen_agentchat.agents import CodeExecutorAgent
from autogen_ext.code_executors import DockerCommandLineCodeExecutor


async def run_code_executor_agent() -> None:
    # Create a code executor agent that uses a Docker container to execute code.
    code_executor = DockerCommandLineCodeExecutor(work_dir="coding")
    await code_executor.start()
    code_executor_agent = CodeExecutorAgent("code_executor", code_executor=code_executor)

    # Run the agent with a given code snippet.
    task = """Here is some code
```python
print('Hello world')
```
"""
    code_execution_result = await code_executor_agent.run(task=task)
    print(code_execution_result.messages[-1])

    # Stop the code executor.
    await code_executor.stop()


# Use asyncio.run(run_code_executor_agent()) when running in a script.
await run_code_executor_agent()

source='code_executor' models_usage=None content='Hello world\n'


This example shows the agent executing a code snippet that prints "Hello world".
The agent then returns the output of the code execution.

## Build Your Own Agents

You may have agents with behaviors that do not fall into a preset. 
In such cases, you can build custom agents.

All agents in AgentChat inherit from {py:class}`~autogen_agentchat.agents.BaseChatAgent` 
class and implement the following abstract methods and attributes:

- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages`: The abstract method that defines the behavior of the agent in response to messages. This method is called when the agent is asked to provide a response in {py:meth}`~autogen_agentchat.agents.BaseChatAgent.run`. It returns a {py:class}`~autogen_agentchat.base.Response` object.
- {py:meth}`~autogen_agentchat.agents.BaseChatAgent.reset`: The abstract method that resets the agent to its initial state. This method is called when the agent is asked to reset itself.
- {py:attr}`~autogen_agentchat.agents.BaseChatAgent.produced_message_types`: The list of possible message types the agent can produce in its response.

Optionally, you can implement the the {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream` method to stream messages as they are generated by the agent. If this method is not implemented, the agent
uses the default implementation of {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages_stream`
that calls the {py:meth}`~autogen_agentchat.agents.BaseChatAgent.on_messages` method and
yields all messages in the response.


### UserProxyAgent 

A common use case for building a custom agent is to create an agent that acts as a proxy for the user.

In the example below we show how to implement a `UserProxyAgent` - an agent that asks the user to enter
some text through console and then returns that message as a response.

In [ ]:
import asyncio
from typing import List, Sequence

from autogen_agentchat.agents import BaseChatAgent
from autogen_agentchat.base import Response
from autogen_agentchat.messages import (
    ChatMessage,
    StopMessage,
    TextMessage,
)
from autogen_core.base import CancellationToken


class UserProxyAgent(BaseChatAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name, "A human user.")

    @property
    def produced_message_types(self) -> List[type[ChatMessage]]:
        return [TextMessage, StopMessage]

    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        user_input = await asyncio.get_event_loop().run_in_executor(None, input, "Enter your response: ")
        if "TERMINATE" in user_input:
            return Response(chat_message=StopMessage(content="User has terminated the conversation.", source=self.name))
        return Response(chat_message=TextMessage(content=user_input, source=self.name))

    async def reset(self, cancellation_token: CancellationToken) -> None:
        pass


async def run_user_proxy_agent() -> None:
    user_proxy_agent = UserProxyAgent(name="user_proxy_agent")
    user_proxy_agent_result = await user_proxy_agent.run(task="What's your thought?")
    print(user_proxy_agent_result.messages[-1])


# Use asyncio.run(run_user_proxy_agent()) when running in a script.
await run_user_proxy_agent()

source='user_proxy_agent' models_usage=None content='I am glad to be here.'
